<div style="max-width:1400px;margin-center: auto">
<img src="images\constructor.png" width="600"/>
</div>

In [1]:
import findspark
findspark.init()
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local') \
    .appName("constructor") \
    .config("spark.sql.catalogImplementation", "hive") \
    .getOrCreate()

In [2]:
from pyspark.sql.functions import col,current_timestamp

In [3]:
constructors_schema = "constructorId INT, constructorRef INT, name STRING, nationality STRING, url STRING "

In [4]:
constructor_df = spark.read.json(r"E:\unused\Udemy\Spark_practice\raw\raw_files\constructors.json", schema = constructors_schema )
constructor_df.show(2)

+-------------+--------------+----------+-----------+--------------------+
|constructorId|constructorRef|      name|nationality|                 url|
+-------------+--------------+----------+-----------+--------------------+
|            1|          null|   McLaren|    British|http://en.wikiped...|
|            2|          null|BMW Sauber|     German|http://en.wikiped...|
+-------------+--------------+----------+-----------+--------------------+
only showing top 2 rows



In [5]:
# constructor_trans_df = constructor_df.select(col("constructorId").alias("constructor_id"),
#                                              col("constructorRef").alias("constructor_ref"),
#                                              col("name"),
#                                              col("nationality"))
# constructor_trans_df.show(2)

In [6]:
# constructor_drop_df = constructor_df.drop(constructor_df.url)
# constructor_drop_df = constructor_df.drop("url")
# constructor_drop_df = constructor_df.drop(constructor_df["url"])
constructor_drop_df = constructor_df.drop(col("url"))
constructor_drop_df.show(2)

+-------------+--------------+----------+-----------+
|constructorId|constructorRef|      name|nationality|
+-------------+--------------+----------+-----------+
|            1|          null|   McLaren|    British|
|            2|          null|BMW Sauber|     German|
+-------------+--------------+----------+-----------+
only showing top 2 rows



In [7]:
constructor_fnl_df = constructor_drop_df.withColumn("ingestion_date",current_timestamp()).\
                    withColumnRenamed("constructorId","constructor_id").\
                    withColumnRenamed("constructorRef","constructor_ref")

In [8]:
constructor_fnl_df.show(2)

+--------------+---------------+----------+-----------+--------------------+
|constructor_id|constructor_ref|      name|nationality|      ingestion_date|
+--------------+---------------+----------+-----------+--------------------+
|             1|           null|   McLaren|    British|2024-01-17 12:09:...|
|             2|           null|BMW Sauber|     German|2024-01-17 12:09:...|
+--------------+---------------+----------+-----------+--------------------+
only showing top 2 rows



In [9]:
constructor_fnl_df.write.mode('overwrite').format('parquet').saveAsTable('f1_processed.constructors')